In [27]:
# upgrade version of altair from 4.2.0 to 5.0.0
%pip install altair==5.0.0rc1

     -------------------------------------- 709.5/709.5 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: altair
    Found existing installation: altair 4.2.0
    Uninstalling altair-4.2.0:
      Successfully uninstalled altair-4.2.0


In [91]:
import altair as alt
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/bfro_reports_fall2022.csv')
print(alt.__version__)
myJekyllDir = 'C:/Users/m-kor/Workspace/CptK.github.io/assets/json/'
df.columns

5.0.0rc1


Index(['observed', 'location_details', 'county', 'state', 'season', 'title',
       'latitude', 'longitude', 'date', 'number', 'classification', 'geohash',
       'temperature_high', 'temperature_mid', 'temperature_low', 'dew_point',
       'humidity', 'cloud_cover', 'moon_phase', 'precip_intensity',
       'precip_probability', 'precip_type', 'pressure', 'summary', 'uv_index',
       'visibility', 'wind_bearing', 'wind_speed', 'location'],
      dtype='object')

In [93]:
brush = alt.selection_interval()

heatmap = alt.Chart('https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/bfro_reports_fall2022.csv').mark_rect().encode(
    x=alt.X('wind_speed:Q', bin=True),
    y=alt.Y('state:N'),
    color=alt.Color('count(state):Q', scale=alt.Scale(scheme='Blues'))
).properties(
    height=500
).add_params(
    brush
)

bar_chart = alt.Chart.from_dict({
  "description": "Mean number of ufos per state",
  "data": {"url": "https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/bfro_reports_fall2022.csv"
  },
  "height":400,
  "width":500,
  "mark": "bar",
  "encoding": {
    "x": {"field": "state", "type": "nominal"},
    "y": {"aggregate":"mean", "field":"number", "type":"quantitative",
      "axis": {
        "title": "mean number of ufos"
      }
    }
  }
}).transform_filter(
    brush
)

In [94]:
dashbord = heatmap.properties(width=150) | bar_chart.properties(width=500)
dashbord

alt.HConcatChart(...)

In [87]:
dashbord.save(myJekyllDir + 'IS445_assignment10_ufo_dashboard.json')

In [90]:
input_dropdown = alt.binding_select(options=df['state'].unique(), name='Region')
selection = alt.selection_point(fields=['state'], bind=input_dropdown)
color = alt.condition(
    selection,
    alt.Color('state:N').legend(None),
    alt.value('lightgray')
)

us_map = alt.Chart.from_dict({
    "width": 800,
    "height": 500,
    "projection": {
        "type": "albersUsa"
    },
    "data": {
        "url": "https://raw.githubusercontent.com/vega/vega-datasets/master/data/us-10m.json",
        "format": {
            "type": "topojson",
            "feature": "states"
        }
    },
        "mark": {
        "type": "geoshape",
        "fill": "lightgray",
        "stroke": "white"
    }
})

sightings_map = alt.Chart.from_dict({
    "data": {
        "url": "https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/bfro_reports_fall2022.csv"
    },
    "encoding": {
        "longitude": {
            "field": "longitude",
            "type": "quantitative"
        },
        "latitude": {
            "field": "latitude",
            "type": "quantitative"
        }
    },
    "mark": {
        "type": "circle",
    }
}).mark_point().encode(
    color=color,
).add_params(
    selection
)

map_chart = us_map + sightings_map
map_chart

NotImplementedError: condition predicate of type <class 'bool'>

In [88]:
map_chart.properties(width='container').save(myJekyllDir + 'IS445_assignment10_ufo_sightings_map.json')